In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#reading data
movies_df = pd.read_csv('../../Dataset/ml-25m/movies.csv')
ratings_df = pd.read_csv('../../Dataset/ml-25m/ratings.csv')

In [2]:
#making new df for calculating average rating without unnecessary features
df_for_avg_rat = ratings_df.drop(columns=['userId', 'timestamp'])
df_for_avg_rat.head()
#sorted by movieId
df_for_avg_rat.sort_values(by='movieId', inplace=True)
df_for_avg_rat
#creating average and total rating dictionaries
avr_rat = {}
tot_rat = {}
temp = 0
counter = 0
for i in range(len(df_for_avg_rat)):
    if df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[-1] or df_for_avg_rat['movieId'].iloc[i] == df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        counter+=1
    elif df_for_avg_rat['movieId'].iloc[i] != df_for_avg_rat['movieId'].iloc[i+1]:
        temp += df_for_avg_rat['rating'].iloc[i]
        avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter
        tot_rat[df_for_avg_rat['movieId'].iloc[i]] = counter
        temp = 0
        counter = 0
    else:
        break

C:\Users\Burak\AppData\Local\Temp\ipykernel_2760\90104468.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  avr_rat[df_for_avg_rat['movieId'].iloc[i]] = temp/counter


In [3]:
#tot_rat dictionary converted to DF
tot_rat_df = pd.DataFrame.from_dict(tot_rat, orient='index')
tot_rat_df.columns = ['totalRating']
tot_rat_df['movieId'] = tot_rat_df.index
tot_rat_df = tot_rat_df.reindex(columns=['movieId', 'totalRating'])
tot_rat_df

#avr_rat dictionary converted to DF
avr_rat_df = pd.DataFrame.from_dict(avr_rat, orient='index')
avr_rat_df.columns = ['averageRating']
avr_rat_df['movieId'] = avr_rat_df.index
avr_rat_df = avr_rat_df.reindex(columns=['movieId', 'averageRating'])
avr_rat_df

,movieId,averageRating
1,1,3.886706
2,2,3.246703
3,3,3.174185
4,4,2.875502
5,5,3.077490
...,...,...
193874,193874,inf
193876,193876,inf
193878,193878,inf
193880,193880,inf


In [7]:
#deleting the movies that have less than 20 ratings
filtered_trd = tot_rat_df[tot_rat_df['totalRating'] >= 50]
filtered_trd.sort_values(by=['totalRating'])

#merged both DF's
ratingsWoUnusedDF = pd.merge(avr_rat_df, filtered_trd, on='movieId')
ratingsWoUnusedDF.sort_values(by=['movieId'])

,movieId,averageRating,totalRating
0,1,3.886706,68468
1,2,3.246703,27142
2,3,3.174185,15584
3,4,2.875502,2988
4,5,3.077490,15473
...,...,...,...
13265,191351,3.495413,109
13266,191367,2.860000,50
13267,191799,2.969697,99
13268,192283,3.545872,109


CREATING USER TRAINING DATA

In [8]:
#ratings data (is going to be user data later)
ratings_df.head()
#drop timestamp column
ratings_df = ratings_df.drop(columns=['timestamp'])

In [9]:
filtered_movieids = ratingsWoUnusedDF['movieId']
filtered_movieids

0             1
1             2
2             3
3             4
4             5
          ...  
13265    191351
13266    191367
13267    191799
13268    192283
13269    192307
Name: movieId, Length: 13270, dtype: int64

In [10]:
ratings_df = pd.merge(ratings_df, filtered_movieids, on='movieId')
ratings_df

,userId,movieId,rating
0,1,307,3.5
1,6,307,4.0
2,56,307,4.0
3,71,307,5.0
4,84,307,3.0
...,...,...,...
27426243,273950,6588,4.0
27426244,275434,6588,4.0
27426245,275841,6588,3.0
27426246,281456,6588,3.5


In [11]:
movies_df = pd.merge(movies_df, filtered_movieids, on='movieId')

movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
13265,191351,To All the Boys I've Loved Before (2018),Comedy|Romance
13266,191367,The Predator (2018),Action|Adventure|Horror|Sci-Fi
13267,191799,Extinction (2018),Sci-Fi
13268,192283,Crazy Rich Asians (2018),Comedy


In [12]:
movies_df.to_csv('./csv/movie_list_25m.csv', index=False)

In [13]:
genres = movies_df['genres'].str.get_dummies('|')
genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13265,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
13266,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
13267,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
13268,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
genres.drop(columns=['(no genres listed)','Film-Noir','IMAX','Musical','War','Western'], axis=1, inplace=True)
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13265,0,0,0,0,1,0,0,0,0,0,0,1,0,0
13266,1,1,0,0,0,0,0,0,0,1,0,0,1,0
13267,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13268,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [15]:
#merging movieid's with their genres
movies_with_genres = pd.concat([movies_df,genres], axis=1)
movies_with_genres

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13265,191351,To All the Boys I've Loved Before (2018),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
13266,191367,The Predator (2018),Action|Adventure|Horror|Sci-Fi,1,1,0,0,0,0,0,0,0,1,0,0,1,0
13267,191799,Extinction (2018),Sci-Fi,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13268,192283,Crazy Rich Asians (2018),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [16]:
user_df = pd.merge(ratings_df, movies_with_genres, left_on='movieId', right_on='movieId')
user_df.drop(columns=['title','genres'], axis=1, inplace=True)
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,307,3.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0
13995,1,1091,1.5,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20133,1,1257,4.5,0,0,0,0,1,0,0,0,0,0,0,1,0,0
26035,1,1449,4.5,0,0,0,0,1,0,0,0,0,0,0,0,0,0
32902,1,1590,2.5,0,0,0,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22654842,283228,799,3.5,0,0,0,0,1,0,0,0,0,1,0,0,0,1
5837258,283228,1617,5.0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
9939126,283228,5502,3.5,0,0,0,0,0,0,0,0,0,1,0,0,1,1
19609648,283228,2872,4.5,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [17]:
#how many ratings each user gave as a dataframe
rating_count = user_df.pivot_table(index=['userId'], aggfunc='size')
rating_count = pd.DataFrame(rating_count)
rating_count.columns = ['rating count']
rating_count['userId'] = rating_count.index
rating_count.index.name = None
rating_count

,rating count,userId
1,16,1
2,15,2
3,11,3
4,736,4
5,72,5
...,...,...
283224,329,283224
283225,20,283225
283226,11,283226
283227,17,283227


In [13]:
"""rating_count.sort_values('rating count',inplace=True)
rating_count[:5]
users_for_test = rating_count.sort_values('rating count')[:5]
"""

"rating_count.sort_values('rating count',inplace=True)\nrating_count[:5]\nusers_for_test = rating_count.sort_values('rating count')[:5]\n"

In [13]:
"""users_for_test"""

'users_for_test'

In [14]:
"""rating_count.drop(index=[175,578,324,320,397], inplace=True)
rating_count.sort_values('userId',inplace=True)
rating_count"""

"rating_count.drop(index=[175,578,324,320,397], inplace=True)\nrating_count.sort_values('userId',inplace=True)\nrating_count"

In [15]:
"""#merge rating count with user_df
test_user_df = pd.merge(user_df, users_for_test, left_on='userId', right_on='userId')
test_user_df.sort_values(by=['userId'])"""

"#merge rating count with user_df\ntest_user_df = pd.merge(user_df, users_for_test, left_on='userId', right_on='userId')\ntest_user_df.sort_values(by=['userId'])"

In [18]:
#merge rating count with user_df
user_df = pd.merge(user_df, rating_count, left_on='userId', right_on='userId')
user_df.sort_values(by=['userId'])

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,307,3.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16
15,1,3893,3.5,0,0,0,0,1,1,0,1,0,0,0,1,0,1,16
13,1,3698,3.5,1,0,0,0,0,0,0,0,0,0,0,0,1,0,16
12,1,3424,4.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16
11,1,3020,4.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13912837,283228,946,4.5,0,0,0,0,1,0,0,1,0,0,0,0,0,0,113
13912838,283228,1077,4.5,0,0,0,0,1,0,0,0,0,0,0,0,1,0,113
13912839,283228,1284,5.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,113
13912825,283228,3114,4.5,0,1,1,1,1,0,0,0,1,0,0,0,0,0,113


In [19]:
#getting column names for the for loop below
columnnames = user_df.columns
columnnames

Index(['userId', 'movieId', 'rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'rating count'],
      dtype='object')

In [20]:
#multiplying every rating by genres to find average rating each user gave for each genre
for i in range(3,17):
    user_df[columnnames[i]] = user_df[columnnames[i]] * user_df['rating']
user_df.sort_values('userId', inplace=True)
user_df


,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,307,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,16
15,1,3893,3.5,0.0,0.0,0.0,0.0,3.5,3.5,0.0,3.5,0.0,0.0,0.0,3.5,0.0,3.5,16
13,1,3698,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,16
12,1,3424,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,16
11,1,3020,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13912837,283228,946,4.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,113
13912838,283228,1077,4.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,113
13912839,283228,1284,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,113
13912825,283228,3114,4.5,0.0,4.5,4.5,4.5,4.5,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,113


In [21]:
#replacing zero values with nan because we don't want zero value counts to affect average
user_df.replace(0, np.nan, inplace=True)
#grouping by user id so it stays the same
grouped = user_df.groupby(['userId'])

def sum_and_divide(x):
    return x.sum() / x.notnull().sum()

user_data_with_average_ratings = grouped.apply(sum_and_divide)
user_data_with_average_ratings

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,,
1,1.0,2254.062500,3.312500,2.875000,1.500000,NaN,NaN,3.750000,3.000000,NaN,3.666667,4.500000,2.25000,NaN,4.000000,2.750000,2.642857,16.0
2,2.0,1869.266667,3.666667,3.500000,3.500000,NaN,NaN,3.611111,3.500000,NaN,3.812500,NaN,4.00000,NaN,3.700000,3.500000,3.500000,15.0
3,3.0,1604.363636,3.545455,5.000000,4.000000,NaN,4.000000,3.500000,3.500000,NaN,3.625000,NaN,3.00000,3.500000,NaN,NaN,3.666667,11.0
4,4.0,5476.336957,3.397418,3.619403,3.477612,3.546875,2.468750,2.904930,3.786765,3.416667,3.603734,3.413793,3.44898,3.819149,3.213483,3.474138,3.727273,736.0
5,5.0,22677.527778,4.263889,4.269231,4.285714,NaN,NaN,4.263158,4.338710,4.500000,4.310000,4.500000,NaN,4.318182,4.500000,4.500000,4.250000,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283224,283224.0,907.869301,3.732523,3.724638,3.888889,4.500000,3.555556,3.528846,3.840000,3.666667,3.965517,3.625000,3.44186,3.956522,3.564516,3.869565,3.756098,329.0
283225,283225.0,1774.950000,3.100000,2.850000,2.666667,NaN,NaN,3.136364,3.571429,NaN,2.833333,NaN,NaN,3.500000,3.000000,2.500000,2.928571,20.0
283226,283226.0,1246.181818,1.818182,1.666667,2.000000,NaN,1.000000,1.750000,1.500000,NaN,2.000000,NaN,2.00000,2.000000,1.000000,2.000000,1.750000,11.0


In [22]:
#rewinding the zero values
user_data_with_average_ratings.replace(np.nan, 0, inplace=True)
#dropping movieid column because we don't need it
user_data_with_average_ratings.drop(columns='movieId', axis=1, inplace=True)
user_data_with_average_ratings

,userId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,
1,1.0,3.312500,2.875000,1.500000,0.000000,0.000000,3.750000,3.000000,0.000000,3.666667,4.500000,2.25000,0.000000,4.000000,2.750000,2.642857,16.0
2,2.0,3.666667,3.500000,3.500000,0.000000,0.000000,3.611111,3.500000,0.000000,3.812500,0.000000,4.00000,0.000000,3.700000,3.500000,3.500000,15.0
3,3.0,3.545455,5.000000,4.000000,0.000000,4.000000,3.500000,3.500000,0.000000,3.625000,0.000000,3.00000,3.500000,0.000000,0.000000,3.666667,11.0
4,4.0,3.397418,3.619403,3.477612,3.546875,2.468750,2.904930,3.786765,3.416667,3.603734,3.413793,3.44898,3.819149,3.213483,3.474138,3.727273,736.0
5,5.0,4.263889,4.269231,4.285714,0.000000,0.000000,4.263158,4.338710,4.500000,4.310000,4.500000,0.00000,4.318182,4.500000,4.500000,4.250000,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283224,283224.0,3.732523,3.724638,3.888889,4.500000,3.555556,3.528846,3.840000,3.666667,3.965517,3.625000,3.44186,3.956522,3.564516,3.869565,3.756098,329.0
283225,283225.0,3.100000,2.850000,2.666667,0.000000,0.000000,3.136364,3.571429,0.000000,2.833333,0.000000,0.00000,3.500000,3.000000,2.500000,2.928571,20.0
283226,283226.0,1.818182,1.666667,2.000000,0.000000,1.000000,1.750000,1.500000,0.000000,2.000000,0.000000,2.00000,2.000000,1.000000,2.000000,1.750000,11.0


In [23]:
#get rating count a list for repeating process
rating_count_as_list = user_data_with_average_ratings["rating count"].values.tolist()
rating_count_as_list

[16.0,
 15.0,
 11.0,
 736.0,
 72.0,
 42.0,
 15.0,
 31.0,
 1.0,
 121.0,
 19.0,
 18.0,
 20.0,
 174.0,
 162.0,
 41.0,
 2.0,
 92.0,
 260.0,
 4.0,
 15.0,
 26.0,
 17.0,
 6.0,
 16.0,
 105.0,
 16.0,
 24.0,
 9.0,
 17.0,
 71.0,
 15.0,
 1.0,
 82.0,
 66.0,
 149.0,
 33.0,
 141.0,
 92.0,
 10.0,
 2.0,
 308.0,
 246.0,
 7.0,
 122.0,
 100.0,
 140.0,
 170.0,
 8.0,
 11.0,
 388.0,
 30.0,
 127.0,
 38.0,
 280.0,
 761.0,
 30.0,
 25.0,
 75.0,
 82.0,
 16.0,
 29.0,
 16.0,
 14.0,
 32.0,
 15.0,
 127.0,
 39.0,
 6.0,
 94.0,
 233.0,
 179.0,
 465.0,
 19.0,
 140.0,
 56.0,
 15.0,
 4.0,
 425.0,
 15.0,
 1088.0,
 45.0,
 92.0,
 153.0,
 15.0,
 19.0,
 18.0,
 302.0,
 14.0,
 148.0,
 14.0,
 15.0,
 19.0,
 16.0,
 158.0,
 10.0,
 15.0,
 20.0,
 144.0,
 511.0,
 46.0,
 15.0,
 133.0,
 18.0,
 44.0,
 11.0,
 32.0,
 29.0,
 5.0,
 16.0,
 67.0,
 14.0,
 12.0,
 303.0,
 16.0,
 21.0,
 68.0,
 7.0,
 256.0,
 17.0,
 66.0,
 44.0,
 5.0,
 8.0,
 8.0,
 15.0,
 19.0,
 69.0,
 9.0,
 76.0,
 16.0,
 134.0,
 15.0,
 1205.0,
 67.0,
 44.0,
 137.0,
 440.0,
 15.0,
 5.0

In [24]:
#repeat every column by userid according to the the users rating counts
repeated_user_df = user_data_with_average_ratings.loc[user_data_with_average_ratings.index.repeat(rating_count_as_list)]
repeated_user_df.rename(columns={'rating': 'average rating'}, inplace=True)
repeated_user_df


,userId,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
userId,,,,,,,,,,,,,,,,,
1,1.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857,16.0
1,1.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857,16.0
1,1.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857,16.0
1,1.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857,16.0
1,1.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283228,283228.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857,113.0
283228,283228.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857,113.0
283228,283228.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857,113.0


In [25]:
#reposition the columns beacuse it is easier to read
new_column_order = ['userId', 'rating count', 'average rating', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller']
repeated_user_df = repeated_user_df.reindex(columns=new_column_order)
repeated_user_df

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
userId,,,,,,,,,,,,,,,,,
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857


In [26]:
#creating x user training data (dropping userid, ratingcount, average rating)
x_user_train = repeated_user_df
x_user_train

,userId,rating count,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
userId,,,,,,,,,,,,,,,,,
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
1,1.0,16.0,3.312500,2.875000,1.500000,0.00,0.0,3.750000,3.0,0.0,3.666667,4.500000,2.25,0.000000,4.000000,2.750000,2.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857
283228,283228.0,113.0,4.340708,4.240741,4.421053,4.75,4.6,4.319149,4.5,0.0,4.202128,4.277778,4.00,4.652174,4.409091,4.166667,4.342857


CREATING USER TEST DATA

In [27]:
#y train should contain movie ratings
y_train_user = user_df['rating']
y_train_user

0           3.5
15          3.5
13          3.5
12          4.5
11          4.0
           ... 
13912837    4.5
13912838    4.5
13912839    5.0
13912825    4.5
13912795    3.5
Name: rating, Length: 27426248, dtype: float64

CREATING ITEM TRAINING DATA

In [28]:
#checking ratings data. we want it to be like movieId | year | average rating | (genres)
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
13265,191351,To All the Boys I've Loved Before (2018),Comedy|Romance
13266,191367,The Predator (2018),Action|Adventure|Horror|Sci-Fi
13267,191799,Extinction (2018),Sci-Fi
13268,192283,Crazy Rich Asians (2018),Comedy


In [29]:
#subtracting the year from the movie title 
pattern = '\((\d{4})\)'
movies_df['year'] = movies_df.title.str.extract(pattern, expand=False)
movies_df.drop(columns=['title','genres'], axis=1, inplace=True)

Burada bazı filmlerin başlığında film yılı yazmadığı için NaN değer alan year değerleri vardı, filtreleme yapınca kalmadı

Büyük verisetine geçince filtreden sonra 19 filmin yılı NaN olarak görünüyor, aşağıda bunun için işlemler yapacağım

In [30]:
movies_df['year'].isnull().sum()

13

In [31]:
#filmlerin yıl ortalaması 1994 o yüzden NaN değerleri 1994 ile değiştireceğim

movies_df['year'].sort_values()[:-13].astype(float).mean()

1993.9866485630232

In [32]:
movies_df['year'].replace(np.nan, 1994, inplace=True)

In [33]:
#bazı değerler datatype olarak str görünüyordu, o yüzden to_numeric kullanarak hepsi int yapıldı
movies_df['year'] = pd.to_numeric(movies_df['year'])
movies_df['year'].sort_values()

11208    1896
7933     1902
8647     1903
6138     1915
6285     1916
         ... 
13212    2018
13211    2018
13210    2018
13218    2018
13269    2018
Name: year, Length: 13270, dtype: int64

In [34]:
ratings_df.sort_values(by=['rating'])

,userId,movieId,rating
27426247,281631,6588,0.5
24527486,248695,77201,0.5
9029923,48006,4121,0.5
598814,156208,19,0.5
22934053,260559,1389,0.5
...,...,...,...
3500454,174733,592,5.0
17931557,193743,969,5.0
23933007,164725,4700,5.0
17931554,193640,969,5.0


In [35]:
#finding how many unique movieId's
ratings_df['movieId']. nunique()

13270

In [36]:
#finding average rating for each movie
grouped_movies = user_df.groupby(['movieId'])

movie_ave_rating = grouped_movies.apply(sum_and_divide)
movie_ave_rating.drop(columns=['userId', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Horror', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'rating count'], axis= 1, inplace=True)
movie_ave_rating.rename(columns={"movieId": "movieId", "rating": "average rating"}, inplace=True)
movie_ave_rating.index.name = None
movie_ave_rating

,movieId,average rating
1,1.0,3.886649
2,2.0,3.246583
3,3.0,3.173981
4,4.0,2.874540
5,5.0,3.077291
...,...,...
191351,191351.0,3.463636
191367,191367.0,2.803922
191799,191799.0,2.940000
192283,192283.0,3.513636


In [37]:
#merge average rating with movies_df
movies_df = pd.merge(movies_df, movie_ave_rating)
#x_item_train = pd.merge(movies_df[['movieId', 'year']], movie_ave_rating[['average rating']], on='movieId')
movies_df

,movieId,year,average rating
0,1,1995,3.886649
1,2,1995,3.246583
2,3,1995,3.173981
3,4,1995,2.874540
4,5,1995,3.077291
...,...,...,...
13265,191351,2018,3.463636
13266,191367,2018,2.803922
13267,191799,2018,2.940000
13268,192283,2018,3.513636


In [38]:
movies_with_genres


,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13265,191351,To All the Boys I've Loved Before (2018),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,1,0,0
13266,191367,The Predator (2018),Action|Adventure|Horror|Sci-Fi,1,1,0,0,0,0,0,0,0,1,0,0,1,0
13267,191799,Extinction (2018),Sci-Fi,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13268,192283,Crazy Rich Asians (2018),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [39]:
#creating not repeated item dataframe
x_item_train_not_repeated = pd.merge(movies_df,movies_with_genres)
x_item_train_not_repeated.drop(columns=['title', 'genres'], axis= 1, inplace=True)
x_item_train_not_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,1,1995,3.886649,0,1,1,1,1,0,0,0,1,0,0,0,0,0
1,2,1995,3.246583,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,3,1995,3.173981,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,4,1995,2.874540,0,0,0,0,1,0,0,1,0,0,0,1,0,0
4,5,1995,3.077291,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13265,191351,2018,3.463636,0,0,0,0,1,0,0,0,0,0,0,1,0,0
13266,191367,2018,2.803922,1,1,0,0,0,0,0,0,0,1,0,0,1,0
13267,191799,2018,2.940000,0,0,0,0,0,0,0,0,0,0,0,0,1,0
13268,192283,2018,3.513636,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [40]:
user_df

,userId,movieId,rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller,rating count
0,1,307,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,16
15,1,3893,3.5,NaN,NaN,NaN,NaN,3.5,3.5,NaN,3.5,NaN,NaN,NaN,3.5,NaN,3.5,16
13,1,3698,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,16
12,1,3424,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,16
11,1,3020,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13912837,283228,946,4.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,113
13912838,283228,1077,4.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,113
13912839,283228,1284,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,113
13912825,283228,3114,4.5,NaN,4.5,4.5,4.5,4.5,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,113


In [41]:
#we need to merge according to movieid order in user_df. using locate to get the order of the movieids
y_train_sort = user_df.loc[:, ['movieId', 'rating']]
y_train_sort

,movieId,rating
0,307,3.5
15,3893,3.5
13,3698,3.5
12,3424,4.5
11,3020,4.0
...,...,...
13912837,946,4.5
13912838,1077,4.5
13912839,1284,5.0
13912825,3114,4.5


In [42]:
#merging
x_item_train_repeated = pd.merge(y_train_sort, x_item_train_not_repeated, on='movieId')
x_item_train_repeated.drop(columns=['rating'],inplace=True)
x_item_train_repeated

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27426243,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426244,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426245,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426246,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [43]:
#dropping moiveid because it's not gonna be needed
x_train_item = x_item_train_repeated
x_train_item

,movieId,year,average rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,307,1993,3.971727,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27426243,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426244,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426245,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1
27426246,6588,2002,3.323077,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [39]:
"""x_train_item.drop(columns=['year'],inplace=True)
x_train_item"""

"x_train_item.drop(columns=['year'],inplace=True)\nx_train_item"

In [44]:
x_train_item.to_csv('./csv/x_train_item_25m.csv', index=False, header=False)
x_user_train.to_csv('./csv/x_train_user_25m.csv', index=False, header=False)
y_train_user.to_csv('./csv/y_train_25m.csv', index=False, header=False)

In [45]:
x_item_train_not_repeated.to_csv('./csv/item_vecs_25m.csv', index=False, header=False)

In [46]:
user_movie_rating = user_df[['userId','movieId','rating']]
user_movie_rating

,userId,movieId,rating
0,1,307,3.5
15,1,3893,3.5
13,1,3698,3.5
12,1,3424,4.5
11,1,3020,4.0
...,...,...,...
13912837,283228,946,4.5
13912838,283228,1077,4.5
13912839,283228,1284,5.0
13912825,283228,3114,4.5


In [47]:
inner_dict_name = 'movies'
def create_inner_dict(group):
    return {inner_dict_name: dict(zip(group['movieId'], group['rating']))}

umr_dict = user_movie_rating.groupby('userId').apply(create_inner_dict).to_dict()

In [48]:
umr_dict

{1: {'movies': {307: 3.5,
   3893: 3.5,
   3698: 3.5,
   3424: 4.5,
   3020: 4.0,
   2986: 2.5,
   2840: 3.0,
   2478: 4.0,
   3826: 2.0,
   1591: 1.5,
   1590: 2.5,
   1449: 4.5,
   1257: 4.5,
   1091: 1.5,
   481: 3.5,
   2134: 4.5}},
 2: {'movies': {849: 3.5,
   3363: 4.0,
   2915: 3.5,
   2746: 4.0,
   2707: 3.5,
   2352: 4.0,
   1962: 4.0,
   2108: 3.5,
   1663: 3.0,
   1296: 4.5,
   1244: 3.0,
   170: 3.5,
   2243: 4.5,
   1235: 3.0,
   1186: 3.5}},
 3: {'movies': {828: 4.0,
   2028: 5.0,
   2024: 3.0,
   1985: 2.0,
   1825: 3.0,
   1645: 4.0,
   1321: 4.0,
   1221: 4.0,
   960: 3.0,
   640: 3.0,
   3171: 4.0}},
 4: {'movies': {788: 1.0,
   2791: 4.0,
   2770: 2.5,
   2764: 4.5,
   2763: 4.5,
   432: 2.5,
   434: 1.0,
   435: 2.0,
   2792: 3.5,
   440: 4.0,
   2794: 3.5,
   2803: 3.5,
   1036: 4.5,
   1020: 3.5,
   1015: 0.5,
   941: 4.0,
   924: 4.0,
   919: 4.0,
   879: 2.5,
   875: 3.5,
   2797: 1.5,
   858: 4.0,
   836: 2.5,
   832: 4.0,
   818: 0.5,
   2860: 3.5,
   2858: 4.

In [49]:
with open('umr_dict_25m.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(umr_dict, f)